In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns

import plotly
plotly.tools.set_credentials_file(username='vvr', api_key='Ker155UU70wk3Ccz6iyM')
import plotly.plotly as py
import plotly.graph_objs as go

color = sns.color_palette("husl", 8)
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer


%matplotlib inline

import squarify

import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
#from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
import re

In [2]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
data.append(test, ignore_index=True)
data.sample(3)

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning:

Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.



,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
16761,p217525,b9cf98d322001d5b0ad0312c0c4db4f2,Mrs.,IN,2016-09-24 11:26:27,Grades PreK-2,Health & Sports,"Gym & Fitness, Health & Wellness",On The Move In Our STEM Classroom,My students are 5 and 6 years old and love to ...,My students love the fact that we are a STEM s...,NaN,NaN,My students need tools like virtual reality eq...,3,1
34254,p259271,4b388ba2c0118d1a5191f1dc4dd008a8,Mrs.,WA,2016-07-26 14:54:41,Grades PreK-2,"Applied Learning, Literacy & Language","Early Development, Literacy",Tools for Multisensory Learning,I work with first grade students in a high pov...,Your donations to this project will help my st...,NaN,NaN,My students need kinetic sensory sand and cont...,23,1
18703,p214606,f5b3aa36620afbed67135d8dc7420a94,Ms.,NY,2016-06-08 18:13:30,Grades 3-5,"Health & Sports, Literacy & Language","Health & Wellness, Literacy",New Year...Much Needed Tools To Get Started,My classroom demographic is diverse. My studen...,Students will be able to use colored pencils a...,NaN,NaN,"My students need a dry erase easel board, card...",66,1


In [3]:
data['count_1'] = data['project_essay_1'].apply(lambda x : len(str(x).split(' ')))
data['count_2'] = data['project_essay_2'].apply(lambda x : len(str(x).split(' ')))
data['count_3'] = data['project_essay_3'].apply(lambda x : len(str(x).split(' ')))
data['count_4'] = data['project_essay_4'].apply(lambda x : len(str(x).split(' ')))
data['count_s'] = data['project_resource_summary'].apply(lambda x : len(str(x).split(' ')))

In [4]:
data['project_essay_3'].fillna('UK',inplace = True)
data['project_essay_4'].fillna('UK',inplace = True)

In [5]:
data['essay'] = data['project_essay_1'] + data['project_essay_2'] + data['project_essay_3'] +data['project_essay_4'] + data['project_resource_summary']


In [6]:
data.drop(['project_essay_1', 'project_essay_2', 'project_essay_3', 'project_essay_4','project_resource_summary'], axis =1, inplace = True)


In [7]:
def pir(data):
    t = data.text.str.split(expand=True).stack()
    return t.loc[t.str.len() >= 4].groupby(level=0).apply(' '.join)
data['essay'] = data['essay'].str.findall('\w{4,}').str.join(' ')
#data.sample()

In [8]:
stop = stopwords.words('english')
#data['essay'].str.findall('\w{4,}').str.join(' ')
data['essay'] = data['essay'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#data['essay'].sample()

In [9]:
data['count'] = data['essay'].str.len()

In [10]:
data['project_submitted_datetime'] = pd.to_datetime(data['project_submitted_datetime']) 
data['project_submitted_datetime'] = data['project_submitted_datetime'].map(lambda x: x.strftime('%Y-%m-%d'))


In [11]:
freq = pd.Series(' '.join(data['essay']).split()).value_counts()

In [12]:
freq2 = freq.tail(65000)
data['essay'] = data['essay'].apply(lambda x: ' '.join([word for word in x.split() if word not in (freq2)]))

In [13]:
from textblob import TextBlob
from textblob import Word
#nltk.download('wordnet')
data['essay'].apply(lambda x : ' '.join([Word(word).lemmatize() for word in x.split()]))
data['essay'].head()

0    Most kindergarten students come income househo...
1    elementary school culturally rich school diver...
2    Hello name teach grade Ascent Academy Utah won...
3    students greatest students socially economical...
4    students athletes students interested health p...
Name: essay, dtype: object

In [24]:
'''
data = data.drop(['project_is_approved','project_title'], axis =1)
y = data['project_is_approved']
'''

"\ndata = data.drop(['project_is_approved','project_title'], axis =1)\ny = data['project_is_approved']\n"

In [25]:
data.drop('project_title', inplace  = True, axis =1)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [26]:

# Label encoding
#copied
#thanks
#https://www.kaggle.com/jmbull/xtra-credit-xgb-lgb-w-tfidf-feature-stacking


cols = [
    'teacher_id', 
    'teacher_prefix', 
    'school_state', 
    'project_grade_category', 
    'project_subject_categories', 
    'project_subject_subcategories'
]

for c in (cols):
    le = LabelEncoder()
    le.fit(data[c].astype(str))
    
    X_train[c] = le.transform(X_train[c].astype(str))
    X_test[c] = le.transform(X_test[c].astype(str))
    



"\n\ncols = [\n    'teacher_id', \n    'teacher_prefix', \n    'school_state', \n    'project_grade_category', \n    'project_subject_categories', \n    'project_subject_subcategories'\n]\n\nfor c in (cols):\n    le = LabelEncoder()\n    le.fit(data[c].astype(str))\n    \n    X_train[c] = le.transform(X_train[c].astype(str))\n    X_test[c] = le.transform(X_test[c].astype(str))\n    \n\n"

In [27]:
# Label encoding
#copied
#thanks
#https://www.kaggle.com/jmbull/xtra-credit-xgb-lgb-w-tfidf-feature-stacking
'''

cols = [
    'teacher_id', 
    'teacher_prefix', 
    'school_state', 
    'project_grade_category', 
    'project_subject_categories', 
    'project_subject_subcategories'
]

for c in (cols):
    le = LabelEncoder()
    le.fit(data[c].astype(str))
    
    data[c] = le.transform(data[c].astype(str))
    #data[c] = le.transform(data[c].astype(str))
    
'''

In [28]:
'''
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
X_train['essay'] =  v.fit_transform(X_train['essay'])
X_test['essay']  = v.fit_transform(X_test['essay'])
'''

"\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nv = TfidfVectorizer()\nX_train['essay'] =  v.fit_transform(X_train['essay'])\nX_test['essay']  = v.fit_transform(X_test['essay'])\n"

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
data['essay'] =  v.fit_transform(data['essay'])
#X_test['essay']  = v.fit_transform(X_test['essay'])

In [36]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
'''
target = 'project_is_approved'
IDcol = 'id'

def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
   
    print(metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))
    print(metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
    pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)

    
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

predictors = [x for x in X.columns if x not in ['project_is_approved','id']]
modelfit(xgb1, data, predictors)

'''

"\ntarget = 'project_is_approved'\nIDcol = 'id'\n\ndef modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):\n    \n    if useTrainCV:\n        xgb_param = alg.get_xgb_params()\n        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)\n        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,\n            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)\n        alg.set_params(n_estimators=cvresult.shape[0])\n    \n    #Fit the algorithm on the data\n    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')\n        \n    #Predict training set:\n    dtrain_predictions = alg.predict(dtrain[predictors])\n    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]\n        \n    #Print model report:\n   \n    print(metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))\n    print(metrics.roc_auc_score(dtrai

In [39]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score

X_train, y_train = make_classification()
X_test, y_test = make_classification()
clf = RandomForestClassifier(max_depth=6, random_state=0)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)



from sklearn import metrics
cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_macro')



Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x00000263D0F98198>>
Traceback (most recent call last):
  File "c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py", line 368, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'
Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x00000263D10656D8>>
Traceback (most recent call last):
  File "c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py", line 368, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'
Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x00000263D1089208>>
Traceback (most recent call last):
  File "c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py", line 368, in __del__
    if self.handle is not None:


array([0.8989899 , 0.8       , 0.94987469, 0.8       , 0.9       ])

In [51]:
clf.score(X_test, y_test)

0.55

In [57]:
from xgboost.sklearn import XGBRegressor 
from xgboost.sklearn import XGBRegressor
import scipy.stats as st

xclas = XGBClassifier()  # and for classifier  
xclas.fit(X_train, y_train)  
xclas.predict(X_test)
cross_val_score(xclas, X_train, y_train) 
xclas.score(X_test,y_test)

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: D

0.48

In [52]:
#print(clf.score(X_test, y_test))

one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

params = {  
    "n_estimators": st.randint(3, 40),
    "max_depth": st.randint(3, 40),
    "learning_rate": st.uniform(0.05, 0.4),
    "colsample_bytree": one_to_left,
    "subsample": one_to_left,
    "gamma": st.uniform(0, 10),
    'reg_alpha': from_zero_positive,
    "min_child_weight": from_zero_positive,
}

xgbreg = XGBRegressor(nthreads=-1)  


from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(xgbreg, params, n_jobs=1)  
gs.fit(X_train, y_train)  
#gs.best_model_  
gs.predict(X_test) 
cross_val_score(gs, X_train, y_train)  
gs.score(X_test, y_test)

-0.07985375346596935

In [ ]:
'''
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
   
    print(metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))
    print(metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))


    
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

#predictors = [x for x in X_train.columns if x not in ['ID']]
modelfit(xgb1, X_train, y_train)
'''

In [ ]:
'''
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

#predictors = [x for x in X_train.columns if x not in ['ID']]
modelfit(xgb1, X_train, y_train)
'''